In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import json
from collections import Counter

# Question 3

In [2]:
def clean_plot(plot):
    """ plot -> string type"""
    """ return processed plot -> string """
    
    #tokenisation
    word_tokens = word_tokenize(plot) 
    # stopwords
    stop_words = set(stopwords.words('english'))
    filter_stopsword = [w for w in word_tokens if not w in stop_words]
    # punctuations
    filter_punc = [w for w in filter_stopsword if w.isalnum()]
    # stemming
    ps = PorterStemmer() 
    filter_stem = [ps.stem(w) for w in filter_punc]
    # single string
    clean_plot = " ".join(filter_stem)
    
    return clean_plot

In [ ]:
# We do the preprocessing and the creation of the vocabulary in one go
word_list = []

# start looping over the tsv files
n = 26885+1 # the total nb of articles we have
for x in range(1,n):
    # article_
    data = pd.read_csv("html_tsv_test_"+str(x)+".tsv", sep = "\t", encoding = "utf-8")
    plot = data['Plot'].item()
    
    # replace the old plot with the new one
    data["Plot"] = data["Plot"].replace(data['Plot'].item(),clean_plot(plot))
    
    # ideally we will save over the old file
    data.to_csv("article_"+str(x)+".tsv", sep = '\t')
    
    # THIS PART TO CREATE THE VOCABULARY
    word_list.append(clean_plot(plot).split())

# the Vocabulary is a list of list, we flatten
flat_vocabulary = [item for sublist in word_list for item in sublist]
# putting the elements in a dico will eliminate the duplicates
vocabulary = dict.fromkeys(flat_vocabulary)

# We get the "integers" we are asked
i = 0
for k,v in vocabulary.items():
    vocabulary[k] = i
    i = i +1


In [ ]:
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)
with open("vocabulary.json","w",encoding="utf-8") as outfile:
    outfile.write(json.dumps(vocabulary))

In [3]:
json_file = open("vocabulary.json", "r")
vocabulary = json.load(json_file)

In [50]:
# CVS make it faster than opening and closing each time
df = pd.read_csv(r"output.csv", nrows=1000)

df["Tokens"] = df['Plot'].apply(lambda x: word_tokenize(clean_plot(x)))
df["Date"] = df["Date"].apply(lambda x: str(x).split()[-1])
df["Char_Tokens"] = df["Setting"].apply(lambda x: word_tokenize(str(x)))

In [ ]:
empty_voca={}

for k,v in vocabulary.items():
    empty_voca.update({k:[]})

for index, tokens in enumerate(df["Tokens"]):
    for token in tokens:
        empty_voca[token].append("article_"+str(index+1))

values = [x for x in empty_voca.values()]
index_1 = {v: k for k, v in vocabulary.items()}

i = 0
for k,v in index_1.items():
    index_1.update({k:values[i]})
    i += 1
index_1

In [42]:
def query_input_1(query):
    
    # PLOT PART
    # we clean the input
    query_words = clean_plot(query)
    query_words = query_words.split()
    
    word_to_int = []
    for word in query_words:
        if word in vocabulary:
            word_to_int.append(vocabulary[word])
    
    sets = []
    for y in word_to_int:
        if y in index_1:
            sets.append(set(index_1[y]))
    
    sets = [*set.intersection(*sets)]
    
    return sets

In [123]:
def nssf(article, q_year, q_char, q_rating):
    
    """ 1+(Nb common characters)  * Exp ^ + 1 / (year asked – year actual book)) + 1/(rating count - actual rating) """
    
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/fuck_no_empty/"
    article = open(path+article+".tsv", "r")
    
    data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
    
    data["Char_Tokens"] = data["Setting"].apply(lambda x: word_tokenize(str(x)))

    q_char = set([x.lower() for x in q_char])
    com_char = 0
    for index,tokens in enumerate(data["Char_Tokens"]):
        intersect = q_char.intersection(set([y.lower() for y in tokens]))
        com_char = len(intersect)
            
    data["Date"] = data["Date"].apply(lambda x: str(x).split()[-1])
    year_diff = abs(int(data["Date"].iloc[0]) - int(q_year))
    
    rating_diff = abs(float(data["ratingValue"].iloc[0]) - int(q_rating))
        
    new_score = 1 + com_char * math.exp((1/year_diff)+(1/(rating_diff)))
    
    return new_score


In [124]:
def get_output(articles_set, q_year, q_char, q_rating):
    
    df = pd.DataFrame(columns = ["bookTitle","Plot","URL","Similarity"])
    
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/fuck_no_empty/"
    
    for article in articles_set:
        
        article_doc = open(path+article+".tsv", "r",encoding="utf-8")
        data = pd.read_csv(article_doc, sep = "\t", encoding = "utf-8")

        title = data["bookTitle"].iloc[0]
        plot = data["Plot"].iloc[0]
        url = data["URL"].iloc[0]
        similarity = nssf(article, q_year, q_char, q_rating)

        to_append = [title, plot, url, similarity]
        df_length = len(df)
        df.loc[df_length] = to_append
    
    return df.drop_duplicates().sort_values(by="Similarity",ascending=False)

In [125]:
query = "potter"
q_year = 2000
q_char = ["Sirius", "Harry"]
q_rating = 4.2
get_output(query_input_1(query) ,q_year, q_char, q_rating)

,bookTitle,Plot,URL,Similarity
6,Harry Potter and the Chamber of Secrets,Ever since Harry Potter had come home for the ...,https://www.goodreads.com/book/show/15881.Harr...,28.815166
0,Harry Potter and the Goblet of Fire,Harry Potter is midway through his training as...,https://www.goodreads.com/book/show/6.Harry_Po...,20.665414
4,Harry Potter and the Prisoner of Azkaban,"For twelve long years, the dread fortress of A...",https://www.goodreads.com/book/show/5.Harry_Po...,15.843071
3,Harry Potter and the Sorcerer's Stone,Harry Potter's life is miserable. His parents ...,https://www.goodreads.com/book/show/3.Harry_Po...,12.716439
2,Harry Potter and the Deathly Hallows,Harry Potter is leaving Privet Drive for the l...,https://www.goodreads.com/book/show/136251.Har...,12.575694
5,Harry Potter and the Half-Blood Prince,The war against Voldemort is not going well; e...,https://www.goodreads.com/book/show/1.Harry_Po...,7.828142
10,Harry Potter Series Box Set,"Over 4000 pages of Harry Potter and his world,...",https://www.goodreads.com/book/show/862041.Har...,1.000000
